In [87]:
import psycopg2

connection = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='*********',
    dbname='classicmodels',
)
cursor = connection.cursor()  

In [ ]:
# updating database 
cursor.execute("UPDATE employees SET officecode = 7 WHERE lastname = 'Bott' AND firstname = 'Larry'")

cursor.execute("UPDATE customers SET creditlimit = 60000 WHERE customername = 'Tekni Collectables Inc.'")

cursor.execute("UPDATE products SET quantityinstock = 0 WHERE productline = 'Trucks and Buses'")

cursor.execute("UPDATE orderdetails SET quantityordered = 60 WHERE productcode = 'S18_2957' AND ordernumber = 10103")

In [ ]:
# deleting data from database
cursor.execute("SELECT * FROM customers WHERE customername  LIKE 'Tokyo Collectables, Ltd'")

cursor.execute("SELECT * FROM orders WHERE customernumber = (SELECT customernumber FROM customers WHERE customername LIKE 'Tokyo Collectables, Ltd')")

cursor.execute("DELETE FROM orderdetails WHERE ordernumber IN (SELECT ordernumber FROM orders WHERE customernumber = (SELECT customernumber FROM customers WHERE customername LIKE 'Tokyo Collectables, Ltd'))")

cursor.execute("DELETE FROM orders WHERE customernumber = (SELECT customernumber FROM customers WHERE customername LIKE 'Tokyo Collectables, Ltd')")

cursor.execute("DELETE FROM payments WHERE customernumber = (SELECT customernumber FROM customers WHERE customername LIKE 'Tokyo Collectables, Ltd')")

cursor.execute("DELETE FROM customers WHERE customername  LIKE 'Tokyo Collectables, Ltd'")

In [ ]:
# many-to-many relation
cursor.execute("SELECT O.orderNumber, P.productName, OD.quantityOrdered, P.productname FROM orders O JOIN orderdetails OD ON O.ordernumber = OD.ordernumber JOIN products P ON P.productcode = OD.productcode ")
#cursor.execute("select * from employees")
for row in cursor:
    print(f'{row[0]:5} | {row[1]:45} | {row[2]:3} | {row[3]:6}')
    #print(row)

In [121]:
# aggregation functions

cursor.execute("SELECT ordernumber, SUM(quantityordered) AS Quantity, SUM(quantityordered * priceEach) FROM orderdetails GROUP BY ordernumber ORDER BY ordernumber LIMIT 10")
print(f'Order number     | Quantity ordered   | Total value  ')
print(f'------------------------------------------------------')
for row in cursor:
    print(f'{row[0]:16} | {row[1]:18} | {round(row[1],2):12} ')
print()
print()

cursor.execute("SELECT productline, COUNT(productname) AS Products , AVG(buyprice) AS Average_price  FROM products GROUP BY productline")
print(f'Product line     | Count of products  | Average buy price ')
print(f'------------------------------------------------------------')
for row in cursor:
    print(f'{row[0]:16} | {round(row[1],2):18} | {round(row[2],2):12}')
print()
print()


cursor.execute("SELECT country, MAX(creditlimit), MIN(creditlimit) FROM customers GROUP BY country ORDER BY country DESC LIMIT 5")
print(f'Country          | Max credit limit   | Min credit limit    ')
print(f'------------------------------------------------------------')
for row in cursor:
    print(f'{row[0]:16} | {round(row[1]):18} | {round(row[1]):16}')
print()

Order number     | Quantity ordered   | Total value  
------------------------------------------------------
           10100 |                151 |       151.00 
           10101 |                142 |       142.00 
           10102 |                 80 |        80.00 
           10103 |                566 |       566.00 
           10104 |                443 |       443.00 
           10105 |                545 |       545.00 
           10106 |                675 |       675.00 
           10107 |                229 |       229.00 
           10108 |                561 |       561.00 
           10109 |                212 |       212.00 


Product line     | Count of products  | Average buy price 
------------------------------------------------------------
Classic Cars     |                 38 |        64.45
Trains           |                  3 |        43.92
Planes           |                 12 |        49.63
Trucks and Buses |                 11 |        56.33
Vintage Cars     

In [120]:
# Aggregation functions in relationships

# What is the total order amount for each production line (sum(quantityordered * priceEach))?

cursor.execute("SELECT PL.productLine, sum(quantityordered * priceEach) from productlines PL JOIN products P ON P.productline = PL.productline JOIN orderdetails OD ON OD.productcode = P.productcode GROUP BY PL.productline ORDER BY PL.productline")
print(f'Customer             | Total order amount   ')
print(f'--------------------------------------------')
for row in cursor:
    print(f'{row[0]:20} | {round(row[1],2):16}')
print()
print()


Customer             | Total order amount   
--------------------------------------------
Classic Cars         |        3827967.4
Motorcycles          |       1105696.85
Planes               |        939282.71
Ships                |        647886.24
Trains               |        184875.24
Trucks and Buses     |       1012728.06
Vintage Cars         |       1781751.38




In [117]:
# What is the total of payments made by each company (the query should also return the company name)?

cursor.execute("SELECT C.customername, sum(P.amount) from payments P JOIN customers C ON P.customernumber = C.customernumber GROUP BY customername ORDER BY customername LIMIT 5")
print(f'Customer                            | Amount paid        ')
print(f'---------------------------------------------------------')
for row in cursor:
    print(f'{row[0]:35} | {round(row[1],2):16}')
print()
print()

Customer                            | Amount paid        
---------------------------------------------------------
AV Stores, Co.                      |        148410.09
Alpha Cognac                        |         60483.36
Amica Models & Co.                  |         82223.23
Anna's Decorations, Ltd             |        137034.22
Atelier graphique                   |         22314.36




In [115]:
# How many items did each company order in total?

cursor.execute("SELECT C.customername, sum(OD.quantityordered) from orderdetails OD JOIN orders O ON O.ordernumber = OD.ordernumber JOIN customers C ON O.customernumber = C.customernumber GROUP BY customername ORDER BY C.customername LIMIT 5")
print(f'Customer                            | Quantity ordered   ')
print(f'---------------------------------------------------------')
for row in cursor:
    print(f'{row[0]:35} | {row[1]:16}')
print()
print()

Customer                            | Quantity ordered   
---------------------------------------------------------
AV Stores, Co.                      |             1778
Alpha Cognac                        |              687
Amica Models & Co.                  |              843
Anna's Decorations, Ltd             |             1469
Atelier graphique                   |              270




In [145]:
# multiple grouping

cursor.execute("SELECT customerName, status, count(status) FROM customers JOIN orders ON orders.customernumber = customers.customernumber GROUP BY customername, status ORDER BY customername LIMIT 10")
print(f'Customer                       |Order status     | Count of status   ')
print(f'--------------------------------------------------------------')
for row in cursor:
    print(f'{row[0]:30} | {row[1]:15} | {row[2]}' )
print()
print()

cursor.execute("SELECT E.officecode, jobtitle, count(jobtitle) FROM employees E JOIN offices O ON E.officecode = O.officecode GROUP BY E.officecode, jobtitle ORDER BY E.officecode")
print(f'Office code  | Job title            | Count of employees   ')
print(f'---------------------------------------------------------')
for row in cursor:
    print(f'{row[0]:12} | {row[1]:20} | {row[2]}' )
print()
print()

Customer                       |Order status     | Count of status   
--------------------------------------------------------------
AV Stores, Co.                 | Shipped         | 3
Alpha Cognac                   | Shipped         | 3
Amica Models & Co.             | Shipped         | 2
Anna's Decorations, Ltd        | Shipped         | 4
Atelier graphique              | Shipped         | 3
Australian Collectables, Ltd   | Shipped         | 2
Australian Collectables, Ltd   | Disputed        | 1
Australian Collectors, Co.     | Shipped         | 5
Australian Gift Network, Co    | Shipped         | 3
Auto Associés & Cie.           | Shipped         | 2


Office code  | Job title            | Count of employees   
---------------------------------------------------------
1            | Sales Rep            | 2
1            | VP Marketing         | 1
1            | VP Sales             | 1
1            | Sales Manager (NA)   | 1
1            | President            | 1
2            | Sa